<a href="https://colab.research.google.com/github/takeisika/group-89-eeg-depression/blob/main/eeg_pred_model_dev_share.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/EEG_128channels_resting_lanzhou_2015.zip -d /content/data

Archive:  /content/drive/MyDrive/EEG_128channels_resting_lanzhou_2015.zip
   creating: /content/data/EEG_128channels_resting_lanzhou_2015/
  inflating: /content/data/EEG_128channels_resting_lanzhou_2015/02010013rest 20150703 1333..mat  
  inflating: /content/data/EEG_128channels_resting_lanzhou_2015/02010012rest 20150626 1026..mat  
  inflating: /content/data/EEG_128channels_resting_lanzhou_2015/02020022rest 20150707 1452..mat  
  inflating: /content/data/EEG_128channels_resting_lanzhou_2015/Multivariate Pattern Analysis of EEG-Based Functional Connectivity A Study on the Identification of Depression.pdf  
  inflating: /content/data/EEG_128channels_resting_lanzhou_2015/02010019rest 20150716 1440..mat  
  inflating: /content/data/EEG_128channels_resting_lanzhou_2015/02030020_rest 20151230 1416.mat  
  inflating: /content/data/EEG_128channels_resting_lanzhou_2015/02010005rest 20150507 0907..mat  
  inflating: /content/data/EEG_128channels_resting_lanzhou_2015/02010022restnew 20150724 14.

# ① Get all .mat files

In [4]:
import os
import glob
FOLDER = "/content/data"
mat_files = glob.glob(os.path.join(FOLDER, "**/*.mat"), recursive=True)
print(f"Found {len(mat_files)} .mat files.")

Found 53 .mat files.


# ② Get subj_id & its label

In [6]:
import os
import re
def get_subj_id_and_label(path):
    filename = os.path.basename(path)
    match = re.search(r"(02\d+)", filename)  # 02010002_... or 02020008_... or 02030002_...
    subj_id = match.group(1)
    label = 1 if subj_id.startswith("0201") else 0  # 0201...→depressed (1), others→not depressed(0)
    return subj_id, label